The first part of our code is a script that generates the model's inputs.

Our study will use Probabilistic Sensitivity Analysis : this means that we consider our parameters to have a certain degree of uncertainty, which is included in the model so that we can obtain a range of possible results.

This model is designed to run for a given number of iterations, simulating disease progression for a different patient each time. In order to represent uncertainty, each patient's individual parameters will be randomly generated, according to a probabilty distribution that is set by the user.

In [ ]:
model_params <- data.frame(
  # age at baseline
  n_age_init = 1,
  # maximum age of follow up
  n_age_max  = 8,
  # discount rate for costs and QALYS 
  d_r = 0.04,
  # number of simulations
  n_sim   = 1000
)

n_t <- model_params$n_age_max - model_params$n_age_init

model_params$n_t <- n_t

# the 4 health states of the model:
v_n <- c("S2", "S3", "D") 

# number of health states 
n_states <- length(v_n) 


In [2]:
library(truncnorm)

default_psa_params <- list(
  # prob Stage 2 -> Stage 3 if untreated
  p_S2_S3_unt = list(
    name = "P(S2 → S3) if untreated",
    def_dis = "beta", 
    def_v1 = "30", 
    def_v2 = "170"
  ),
  
  # rate ratio Stage 2 -> Stage 3 treated/untreated
  hr_S2_S3 = list(
    name = "Hazard Ratio for P(S2 → S3) Treated/Untreated",
    def_dis = "normal", 
    def_v1 = "0.7", 
    def_v2 = "0.3"
  ),
  
  # prob Stage 2 -> Dead
  p_S2_D = list(
    name = "P(S2 → Dead)",
    def_dis = "beta", 
    def_v1 = "1", 
    def_v2 = "1990"
  ),
  
  # rate ratio death Stage 3 vs Stage 2
  hr_S3_D = list(
    name = "Hazard Ratio for death S3/S2",
    def_dis = "normal", 
    def_v1 = "0.1", 
    def_v2 = "0.03"
  ),
  
  # cost p/cycle in state S2
  c_S2 = list(
    name = "Cost per cycle in Stage 2",
    def_dis = "gamma", 
    def_v1 = "100", 
    def_v2 = "20"
  ),
  
  # cost p/cycle in state S3
  c_S3 = list(
    name = "Cost per cycle in Stage 3",
    def_dis = "gamma", 
    def_v1 = "177.8", 
    def_v2 = "22.5"
  ),
  
  # cost p/cycle of treatment in state S2
  c_Trt_S2 = list(
    name = "Cost of treatment in Stage 2",
    def_dis = "fixed", 
    def_v1 = "200", 
    def_v2 = ""
  ),
  
  # cost p/cycle of treatment in state S3 (we stop treating in S3)
  c_Trt_S3 = list(
    name = "Cost of treatment in Stage 3",
    def_dis = "fixed", 
    def_v1 = "0", 
    def_v2 = ""
  ),

  # cost p/cycle in state D (fixed value)
  c_D = list(
    name = "Cost of Death",
    def_dis = "fixed", 
    def_v1 = "0", 
    def_v2 = ""
  ),
  
  # utility when stage 2
  u_S2 = list(
    name = "Utility in Stage 2",
    def_dis = "normal", 
    def_v1 = "1", 
    def_v2 = "0.01"
  ),
  
  # utility when stage 3
  u_S3 = list(
    name = "Utility in Stage 3",
    def_dis = "normal", 
    def_v1 = "0.75", 
    def_v2 = "0.02"
  ),
  
  # utility when dead
  u_D = list(
    name = "Utility when dead",
    def_dis = "fixed", 
    def_v1 = "0", 
    def_v2 = ""
  )
)


In [3]:
f_gen_psa <- function(psa_params, model_params) {
  # Initialize an empty list to store the generated data
  df_list <- list()
  
  # Iterate over each parameter in the psa_params list
  for (param_name in names(psa_params)) {
    param_info <- psa_params[[param_name]]
    dist_type <- param_info$def_dis
    v1 <- as.numeric(param_info$def_v1)
    v2 <- as.numeric(param_info$def_v2)
    
    # Generate samples based on the distribution type
    samples <- switch(dist_type,
      "beta" = rbeta(n = model_params$n_sim, shape1 = v1, shape2 = v2),
      "log_normal" = rlnorm(n = model_params$n_sim, meanlog = log(v1), sdlog = v2),
      "normal" = rnorm(n = model_params$n_sim, mean = v1, sd = v2),
      "gamma" = rgamma(n = model_params$n_sim, shape = v1, rate = 1/v2),
      "fixed" = rep(v1, model_params$n_sim),
      stop("Unsupported distribution type")
    )
    
    # Store the samples in the list with the parameter name
    df_list[[param_name]] <- samples
  }
  
  # Convert the list to a data frame
  df <- as.data.frame(df_list)
  return(df)
}

When we print out the head and dimensions of this PSA data frame, we can see that it contains 1000 rows and 12 columns. Each row corresponds to one of our generated patients, and their parameters all differ slightly.

In [ ]:
df_psa <- f_gen_psa(default_psa_params, model_params)

print(paste("rows:", dim(df_psa)[1], "  columns:", dim(df_psa)[2]))
print(head(df_psa))

In [4]:
# Calculate transition rates and probabilities
calculate_transition_rates <- function(psa_table_row) {
  with(as.list(psa_table_row), {
    r_S2_S3_unt <- -log(1 - p_S2_S3_unt)
    r_S2_S3_trt <- hr_S2_S3 * r_S2_S3_unt
    p_S2_S3_trt <- 1 - exp(-r_S2_S3_trt)
    
    r_S2_D <- -log(1 - p_S2_D)
    r_S3_D <- hr_S3_D * r_S2_D
    p_S3_D <- 1 - exp(-r_S3_D)
    
    list(r_S2_S3_unt = r_S2_S3_unt, r_S2_S3_trt = r_S2_S3_trt, p_S2_S3_trt = p_S2_S3_trt,
         r_S2_D = r_S2_D, r_S3_D = r_S3_D, p_S3_D = p_S3_D)
  })
}

In [ ]:
# Extract parameters for the first row
psa_table_row <- df_psa[1,]

transition_rates <- calculate_transition_rates(psa_table_row)
print(transition_rates)

In [5]:
calculate_discount_weights <- function(model_params) {
  n_t <- model_params$n_t
  d_r <- model_params$d_r
  v_dwe <- v_dwc <- 1 / (1 + d_r) ^ (0:n_t)
  list(v_dwe = v_dwe, v_dwc = v_dwc)
}

In [ ]:
discount_weights <- calculate_discount_weights(model_params)
print(discount_weights)

In [6]:
create_transition_matrices <- function(psa_table_row, transition_rates) {
  with(as.list(psa_table_row), {
    m_P_unt <- m_P_trt <- matrix(0, nrow = n_states, ncol = n_states, dimnames = list(v_n, v_n))
    
    # Untreated
    m_P_unt["S2", "S2"] <- 1 - (p_S2_S3_unt + p_S2_D)
    m_P_unt["S2", "S3"] <- p_S2_S3_unt
    m_P_unt["S2", "D"] <- p_S2_D
    m_P_unt["S3", "S3"] <- 1 - transition_rates$p_S3_D
    m_P_unt["S3", "D"] <- transition_rates$p_S3_D
    m_P_unt["D", "D"] <- 1
    
    # Treated
    m_P_trt["S2", "S2"] <- 1 - (transition_rates$p_S2_S3_trt + p_S2_D)
    m_P_trt["S2", "S3"] <- transition_rates$p_S2_S3_trt
    m_P_trt["S2", "D"] <- p_S2_D
    m_P_trt["S3", "S3"] <- 1 - transition_rates$p_S3_D
    m_P_trt["S3", "D"] <- transition_rates$p_S3_D
    m_P_trt["D", "D"] <- 1
    
    list(m_P_unt = m_P_unt, m_P_trt = m_P_trt)
  })
}

In [ ]:
psa_table_row <- df_psa[1,]

transition_matrices <- create_transition_matrices(psa_table_row, transition_rates)
print(transition_matrices)

In [7]:
simulate_markov_traces <- function(model_params, transition_matrices) {
  n_t <- model_params$n_t
  m_TR_trt <- m_TR_unt <- matrix(NA, nrow = n_t + 1, ncol = n_states, dimnames = list(0:n_t, v_n))
  m_TR_unt[1, ] <- m_TR_trt[1, ] <- c(1, 0, 0)
  
  for (t in 1:n_t) {
    m_TR_unt[t + 1, ] <- m_TR_unt[t, ] %*% transition_matrices$m_P_unt
    m_TR_trt[t + 1, ] <- m_TR_trt[t, ] %*% transition_matrices$m_P_trt
  }
  
  list(m_TR_unt = m_TR_unt, m_TR_trt = m_TR_trt)
}

In [ ]:
markov_traces <- simulate_markov_traces(model_params, transition_matrices)
print(markov_traces)

In [8]:
calculate_costs_qalys <- function(psa_table_row, markov_traces, discount_weights) {
  with(as.list(psa_table_row), {
    v_u <- c(u_S2, u_S3, u_D)
    v_c_trt <- c(c_S2 + c_Trt_S2, c_S3, c_D)
    v_c_unt <- c(c_S2, c_S3, c_D)
    
    v_E_unt <- markov_traces$m_TR_unt %*% v_u
    v_E_trt <- markov_traces$m_TR_trt %*% v_u
    v_C_unt <- markov_traces$m_TR_unt %*% v_c_unt
    v_C_trt <- markov_traces$m_TR_trt %*% v_c_trt
    
    te_unt <- t(v_E_unt) %*% discount_weights$v_dwe
    te_trt <- t(v_E_trt) %*% discount_weights$v_dwe
    tc_unt <- t(v_C_unt) %*% discount_weights$v_dwc
    tc_trt <- t(v_C_trt) %*% discount_weights$v_dwc
    
    results <- c(
      "Cost_NoTrt" = tc_unt,
      "Cost_Trt" = tc_trt,
      "QALY_NoTrt" = te_unt,
      "QALY_Trt" = te_trt,
      "ICER" = (tc_trt - tc_unt) / (te_trt - te_unt)
    )
    return(results)
  })
}

In [ ]:
psa_table_row <- df_psa[1,]
results <- calculate_costs_qalys(psa_table_row, markov_traces, discount_weights)
print(results)

In [9]:
f_hybrid_markov_model <- function(psa_table_row, model_params) {
  environment(create_transition_matrices) <- environment()
  environment(simulate_markov_traces)     <- environment()

  transition_rates <- calculate_transition_rates(psa_table_row)
  discount_weights <- calculate_discount_weights(model_params)
  transition_matrices <- create_transition_matrices(psa_table_row, transition_rates)
  markov_traces <- simulate_markov_traces(model_params, transition_matrices)
  results <- calculate_costs_qalys(psa_table_row, markov_traces, discount_weights)
  return(results)
}

In [16]:

f_wrapper <- function(psa_params, model_params) {
  # need to specify environment of inner functions (to use outer function enviroment)
  # alternatively - define functions within the wrapper function.
  environment(f_gen_psa)               <- environment()
  environment(f_hybrid_markov_model)   <- environment()

  df_psa <- f_gen_psa(psa_params, model_params)

  # the 4 health states of the model:
  v_n <- c("S2", "S3", "D") 
  # number of health states 
  n_states <- length(v_n) 

  n_sim <- model_params$n_sim

  # Initialize matrix of results outcomes
  m_out <- matrix(NaN, 
                  nrow = n_sim, 
                  ncol = 5,
                  dimnames = list(1:n_sim,
                                  c("Cost_NoTrt", "Cost_Trt",
                                    "QALY_NoTrt", "QALY_Trt",
                                    "ICER")))

  # run model for each row of PSA inputs
  for(i in 1:n_sim){

    # store results in row of results matrix
    m_out[i,] <- f_hybrid_markov_model(df_psa[i, ], model_params)

  } # close model loop


  #-- Return results --#

  # convert matrix to dataframe (for plots)
  df_out <- as.data.frame(m_out) 

  # output the dataframe from the function  
  return(df_out) 
}

df_out <- f_wrapper(default_psa_params, model_params)

In [14]:
print(df_out)

     Cost_NoTrt Cost_Trt QALY_NoTrt QALY_Trt          ICER
1      19035.32 19734.44   6.332194 6.350351  3.850478e+04
2      19336.46 18734.77   6.386286 6.592343 -2.920011e+03
3      18739.88 19011.99   6.513603 6.601485  3.096352e+03
4      20409.24 20867.26   6.443884 6.509207  7.011594e+03
5      22601.31 22740.36   6.208845 6.282134  1.897314e+03
6      19429.37 20458.34   6.388764 6.363949 -4.146450e+04
7      18826.97 19916.29   6.445817 6.429668 -6.745728e+04
8      17274.97 18076.16   6.554839 6.583975  2.749715e+04
9      22150.55 22432.87   6.295161 6.385585  3.122152e+03
10     19804.21 20744.82   6.318156 6.308139 -9.390606e+04
11     16456.35 17737.01   6.639797 6.600621 -3.268997e+04
12     19822.56 17901.27   6.342479 6.681139 -5.673221e+03
13     18326.92 16769.94   6.298498 6.731242 -3.597917e+03
14     19551.09 20576.21   6.393317 6.388428 -2.096971e+05
15     20619.09 21161.79   6.174155 6.211744  1.443765e+04
16     18939.84 19679.66   6.339224 6.352826  5.439040e+